# This is the SmartGrid Optimizier notebook that accepts Energy Demand, Supply and Storage data, and then displays a recommendation for maximum optimization and sustainability of the Grid's output.

In [ ]:
#!pip install google-api-python-client
!pip install google-api-python-client google-cloud-aiplatform


#import libraries & dependencies
import os
from google.api_core.client_options import ClientOptions
from google.cloud import aiplatform
from google.protobuf import json_format
from googleapiclient.discovery import build

from google.colab import userdata
from IPython.display import HTML
import ipywidgets as widgets
from IPython.display import display

In [29]:
# Create input widgets
supply_input = widgets.FloatText(description='Supply Capacity (MW):', value=1000)
demand_input = widgets.FloatText(description='Demand (MW):', value=800)
storage_input = widgets.FloatText(description='Storage (MWh):', value=100)
analyze_button = widgets.Button(description='Analyze Grid')
output = widgets.Output()

In [30]:
def analyze_energy_grid(supply_capacity, demand, storage_capacity):
    """Analyzes energy grid parameters and provides recommendations."""
    # Calculate metrics
    reserve_margin = ((supply_capacity - demand) / demand) * 100
    storage_ratio = (storage_capacity / demand) * 100

    # Initialize results
    recommendations = []
    status_color = "#e8f5e9"  # Default is green to indicate stability

    # Analyze reserve margin
    if reserve_margin < 15:
        status_color = "#ffebee"  # Red represents a critical state of the Grid
        if reserve_margin < 0:
            recommendations.append("URGENT: Supply cannot meet demand. Immediate action required.")
        else:
            recommendations.append("Warning: Reserve margin below 15%. Add generation capacity.")
    elif reserve_margin > 30:
        recommendations.append("Supply excess. Consider optimizing resources.")

    # Analyze storage
    if storage_ratio < 10:
        status_color = "#fff3e0"  # Orange indicate a warning
        recommendations.append("Low storage capacity. Consider additional storage.")
    elif storage_ratio < 5:
        status_color = "#ffebee"  # Red represents a critical state of the Grid
        recommendations.append("Critical: Very low storage capacity.")

    if not recommendations:
        recommendations.append("Grid parameters are stable.")

    return reserve_margin, storage_ratio, recommendations, status_color

In [ ]:
def generate_sustainability_recommendation(demand_data, supply_data, storage_data):
    # Initialize my projectid, model and API key
    api_key = userdata.get('SmartGridOpt_key')
    project_id = userdata.get('SmartGridOpt_pid')
    mymodel = "Gemini 1.5 Flash"

    # Initialize the Gemini API client
    aiplatform.init(project=project_id, location="us-central1", api_key=api_key)

    # Set the endpoint for the API
    endpoint = "us-central1-aiplatform.googleapis.com"

    # Create an AIPlatform client with your API key and endpoint
    #client_option = ClientOptions(api_endpoint=endpoint, api_key=api_key)
    aiplatform.init(project=project_id, location="us-central1", api_endpoint=endpoint, api_key=api_key)

    # Constructing the prompt with the provided data
    prompt = f"""
    Given the following data on demand, supply, and storage, generate recommendations focused on sustainability of the Energy Grid.

    Demand Data:
    {demand_data}

    Supply Data:
    {supply_data}

    Storage Data:
    {storage_data}

    Please consider optimizing for minimal environmental impact, efficient resource use, and improved sustainability.
    """

  # Define the model
    model = aiplatform.gapic.ModelServiceClient.model_path(project_id, 'us-central1', mymodel)

  # Define the instance (input)
    instance = {"content": prompt}

  # Make the prediction request
    #response = aiplatform.gapic.PredictRequest()
    response = model.predict(instance)

  # Extract the prediction
    return json_format.MessageToJson(response.predictions[0].parameters)



In [32]:
def on_button_click(b):
    with output:
        output.clear_output()

        # Get values from inputs
        supply = supply_input.value
        demand = demand_input.value
        storage = storage_input.value

        # Perform analysis
        reserve_margin, storage_ratio, recommendations, status_color = analyze_energy_grid(supply, demand, storage)

        # Display results
        display(HTML(f"""
        <div style="padding: 15px; background-color: {status_color}; border-radius: 5px; margin: 10px 0;">
            <h3>Analysis Results</h3>
            <p><strong>Reserve Margin:</strong> {reserve_margin:.1f}%</p>
            <p><strong>Storage Ratio:</strong> {storage_ratio:.1f}%</p>
            <h4>Recommendations:</h4>
            <ul>{''.join([f'<li>{r}</li>' for r in recommendations])}</ul>
        </div>
        """))

In [33]:
# Connect button to function
analyze_button.on_click(on_button_click)

In [34]:
# Display the interface
print("Energy Grid Sustainability Analyzer")
print("----------------------------------")
display(supply_input)
display(demand_input)
display(storage_input)
display(analyze_button)
display(output)

Energy Grid Sustainability Analyzer
----------------------------------


FloatText(value=1000.0, description='Supply Capacity (MW):')

FloatText(value=800.0, description='Demand (MW):')

FloatText(value=100.0, description='Storage (MWh):')

Button(description='Analyze Grid', style=ButtonStyle())

Output()

In [ ]:
# Generate Gemini recommendations
recommendatns = generate_sustainability_recommendation(demand_input, supply_input, storage_input)
print("Sustainability Recommendations:\n", recommendatns)